In [1]:
from gevent import monkey
monkey.patch_all()
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = = = = =

import pandas as pd
from selenium.webdriver import ChromeOptions
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

# = = = = = = = = = = = = = = = = = =

df_menu = pd.read_excel('./menu.xlsx', header=0)
list_url = df_menu['Url'].to_list()
print('总数量：' + str(len(list_url)))
print()

work = Queue()
for url in list_url:
    work.put_nowait(url)

# = = = = = = = = = = = = = = = = = =

df = pd.DataFrame(columns=['Part Number',
                           'Category',
                           'OE',
                           'Vehicle',
                           'Src',
                           'Url',
                           'status'])

# = = = = = = = = = = = = = = = = = =

option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_argument('blink-settings=imagesEnabled=false')
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--proxy-server=http://u431.kdltps.com:15818')
option.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36')
option.page_load_strategy = 'eager'

# = = = = = = = = = = = = = = = = = =

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        status = 'error'
        for _ in range(31):
            try:
                browser = Chrome(service=Service('../../chromedriver.exe'), options=option)
                browser.maximize_window()
                
                # = = = = = = = = = = = = = = = = = =
                
                browser.get(url)
                
                # = = = = = = = = = = = = = = = = = =
                
                wait = WebDriverWait(browser, 60, 0.1, None)
                
                # = = = = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//span[@id="lblProductName"]')))
                part_number = browser.find_element(By.XPATH, '//span[@id="lblProductName"]').text

                # = = = = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//span[@id="lblProductDesc"]')))
                category = browser.find_element(By.XPATH, '//span[@id="lblProductDesc"]').text

                # = = = = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//section[@id="productOE"]/descendant::*/th[@scope="row"]')))
                oe = browser.find_element(By.XPATH, '//section[@id="productOE"]/descendant::*/th[@scope="row"]').text

                # = = = = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                wait.until(EC.presence_of_element_located((By.XPATH, '//iframe[@id="ifDetailApp"]')))
                for _ in range(100):
                    if browser.find_element(By.XPATH, '//iframe[@id="ifDetailApp"]').get_attribute('src') == None:
                        time.sleep(1)
                iframe = browser.find_element(By.XPATH, '//iframe[@id="ifDetailApp"]')
                browser.switch_to.frame(iframe)
                
                # = = = = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//tr[@class="detail-app-row"]')))
                list_row = browser.find_elements(By.XPATH, '//tr[@class="detail-app-row"]')
                for row in list_row:
                    make = row.find_element(By.XPATH, './td[2]').text
                    model = row.find_element(By.XPATH, './td[3]').text
                    year = row.find_element(By.XPATH, './td[1]').text

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: [year]}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = [year]
                        else:
                            dict_vehicle[make][model].append(year)

                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_vehicle.append(make + ' ' + model + ' ' + min(dict_vehicle[make][model]) + '-' + max(dict_vehicle[make][model]))

                vehicle = '\n'.join(list_vehicle)
                browser.switch_to.parent_frame()

                # = = = = = = = = = = = = = = = = = =
                
                wait.until(EC.presence_of_element_located((By.XPATH, '//img[@id="imgProductMain"]')))
                src = browser.find_element(By.XPATH, '//img[@id="imgProductMain"]').get_attribute('src').replace('/icon/', '/medium/')
                
                # = = = = = = = = = = = = = = = = = =
                
                browser.close()
                
                # = = = = = = = = = = = = = = = = = =

                status = 'ok'
                df_temp = pd.DataFrame({'Part Number': part_number,
                                        'Category': category,
                                        'OE': oe,
                                        'Vehicle': vehicle,
                                        'Src': src,
                                        'Url': url,
                                        'status': ['ok']})
                
                break

            except:
                time.sleep(3)
                continue

        # = = = = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame({'Url': url,
                                    'status': ['error']})

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')
        
        print(url + '  -  [' + status + '] - 剩余：' + str(work.qsize()))
        
# = = = = = = = = = = = = = = = = = =

list_task = []
for _ in range(6):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df.to_excel('./part.xlsx', index=False)
print()
print('搞定')

总数量：4



KeyboardInterrupt
2023-06-12T11:32:04Z


KeyboardInterrupt: 